In [150]:
import torch


torch.manual_seed(0)

A = torch.randint(-10, 10,(4,3))
B = torch.randint(-10, 10,(2, 3,2))

A, B

(tensor([[ -6,   9,   3],
         [-10,  -7,   9],
         [ -3,  -7,   7],
         [ -7,  -9,  -4]]),
 tensor([[[ 6,  9],
          [ 8,  6],
          [ 6, -2]],
 
         [[ 4,  3],
          [-4,  9],
          [ 1,  4]]]))

In [151]:
A_exp = A.unsqueeze(1).expand(-1, B.shape[2], -1).unsqueeze(0).expand(B.shape[0],-1,-1,-1)
B_exp = B.transpose(1, 2).unsqueeze(1).expand(-1, A.shape[0], -1, -1)

print(A_exp.shape, B_exp.shape)
A_exp, B_exp

torch.Size([2, 4, 2, 3]) torch.Size([2, 4, 2, 3])


(tensor([[[[ -6,   9,   3],
           [ -6,   9,   3]],
 
          [[-10,  -7,   9],
           [-10,  -7,   9]],
 
          [[ -3,  -7,   7],
           [ -3,  -7,   7]],
 
          [[ -7,  -9,  -4],
           [ -7,  -9,  -4]]],
 
 
         [[[ -6,   9,   3],
           [ -6,   9,   3]],
 
          [[-10,  -7,   9],
           [-10,  -7,   9]],
 
          [[ -3,  -7,   7],
           [ -3,  -7,   7]],
 
          [[ -7,  -9,  -4],
           [ -7,  -9,  -4]]]]),
 tensor([[[[ 6,  8,  6],
           [ 9,  6, -2]],
 
          [[ 6,  8,  6],
           [ 9,  6, -2]],
 
          [[ 6,  8,  6],
           [ 9,  6, -2]],
 
          [[ 6,  8,  6],
           [ 9,  6, -2]]],
 
 
         [[[ 4, -4,  1],
           [ 3,  9,  4]],
 
          [[ 4, -4,  1],
           [ 3,  9,  4]],
 
          [[ 4, -4,  1],
           [ 3,  9,  4]],
 
          [[ 4, -4,  1],
           [ 3,  9,  4]]]]))

In [152]:
squared_diffs = (A_exp - B_exp) ** 2
squared_diffs

tensor([[[[144,   1,   9],
          [225,   9,  25]],

         [[256, 225,   9],
          [361, 169, 121]],

         [[ 81, 225,   1],
          [144, 169,  81]],

         [[169, 289, 100],
          [256, 225,   4]]],


        [[[100, 169,   4],
          [ 81,   0,   1]],

         [[196,   9,  64],
          [169, 256,  25]],

         [[ 49,   9,  36],
          [ 36, 256,   9]],

         [[121,  25,  25],
          [100, 324,  64]]]])

In [153]:
summm = squared_diffs.sum(dim=3)
summm

tensor([[[154, 259],
         [490, 651],
         [307, 394],
         [558, 485]],

        [[273,  82],
         [269, 450],
         [ 94, 301],
         [171, 488]]])

In [154]:
summm = squared_diffs.mean(dim=3, dtype=torch.float32)
summm

tensor([[[ 51.3333,  86.3333],
         [163.3333, 217.0000],
         [102.3333, 131.3333],
         [186.0000, 161.6667]],

        [[ 91.0000,  27.3333],
         [ 89.6667, 150.0000],
         [ 31.3333, 100.3333],
         [ 57.0000, 162.6667]]])

In [173]:
aaa = summm.transpose(1,2)
aaa

tensor([[[ 51.3333, 163.3333, 102.3333, 186.0000],
         [ 86.3333, 217.0000, 131.3333, 161.6667]],

        [[ 91.0000,  89.6667,  31.3333,  57.0000],
         [ 27.3333, 150.0000, 100.3333, 162.6667]]])

In [174]:
y = torch.tensor([[0,1], [2,3]])
y

tensor([[0, 1],
        [2, 3]])

In [175]:
indices_unsqueezed = y.unsqueeze(2)  # Adds a third dimension for compatibility with gather
selected = torch.gather(aaa, 2, indices_unsqueezed).squeeze(2)

selected


tensor([[ 51.3333, 217.0000],
        [ 31.3333, 162.6667]])